## Preparing queries for Abstract Sifter

Querying Pubmed against the PFAS landscape (new inventory) to determine article counts.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import openpyxl
from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage

#Show mols in dataframes
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage
from IPython.core.display import HTML
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
import os

In [2]:
TOP = os.getcwd().replace('notebooks', '')
raw_dir = TOP + 'data/raw/'
interim_dir = TOP + 'data/interim/'
external_dir = TOP + 'data/external/'
figures_dir = TOP + 'reports/figures/'

In [3]:
df = pd.read_excel(interim_dir+'final_universe_categorisations_wCDR_100524.xlsx', index_col = [0])

In [4]:
df = df[['dtxsid','Substance_Name', 'Substance_CASRN']].dropna()

In [15]:
df

,dtxsid,Substance_Name,Substance_CASRN,query,Column3,term1,term2,term3
1,DTXSID90897582,4-(Perfluorononyl)oxybenzenesulfonate sodium,91998-13-9,4-(Perfluorononyl)oxybenzenesulfonate sodium O...,4-(Perfluorononyl)oxybenzenesulfonate sodium O...,4-(Perfluorononyl)oxybenzenesulfonate sodium,91998-13-9,4-(Perfluorononyl)oxybenzenesulfonate sodium
2,DTXSID90896257,(2S)-2-{Bis[4-(heptadecafluorooctyl)phenyl][(t...,914391-37-0,(2S)-2-{Bis[4-(heptadecafluorooctyl)phenyl][(t...,(2S)-2-{Bis[4-(heptadecafluorooctyl)phenyl][(t...,(2S)-2-{Bis[4-(heptadecafluorooctyl)phenyl][(t...,914391-37-0,(2S)-2-{Bis[4-(heptadecafluorooctyl)phenyl][(t...
3,DTXSID90896196,"2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,9-Heptadecaflu...",157584-47-9,"2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,9-Heptadecaflu...","2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,9-Heptadecaflu...","2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,9-Heptadecaflu...",157584-47-9,"2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,9-Heptadecaflu..."
4,DTXSID90896095,"3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,10-Heptadeca...",113823-56-6,"3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,10-Heptadeca...","3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,10-Heptadeca...","3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,10-Heptadeca...",113823-56-6,"3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,10-Heptadeca..."
5,DTXSID90896015,5-(Heptadecafluorooctyl)-1H-imidazole,81769-56-4,5-(Heptadecafluorooctyl)-1H-imidazole OR 81769...,5-(Heptadecafluorooctyl)-1H-imidazole OR 81769...,5-(Heptadecafluorooctyl)-1H-imidazole,81769-56-4,5-(Heptadecafluorooctyl)-1H-imidazole
...,...,...,...,...,...,...,...,...
15403,DTXSID001346201,4‐{[4‐(Dimethylamino)phenyl][4‐ (phenylamino)n...,1310480-28-4,4‐{[4‐(Dimethylamino)phenyl][4‐ (phenylamino)n...,4‐{[4‐(Dimethylamino)phenyl][4‐ (phenylamino)n...,4‐{[4‐(Dimethylamino)phenyl][4‐ (phenylamino)n...,1310480-28-4,4‐{[4‐(Dimethylamino)phenyl][4‐ (phenylamino)n...
15404,DTXSID001346194,4‐{[4‐(Dimethylamino)phenyl][4‐ (ethylamino)na...,1310480-27-3,4‐{[4‐(Dimethylamino)phenyl][4‐ (ethylamino)na...,4‐{[4‐(Dimethylamino)phenyl][4‐ (ethylamino)na...,4‐{[4‐(Dimethylamino)phenyl][4‐ (ethylamino)na...,1310480-27-3,4‐{[4‐(Dimethylamino)phenyl][4‐ (ethylamino)na...
15405,DTXSID001047621,"(4-Methylphenyl)(diphenyl)sulfanium 4,4,5,5,6,...",862261-51-6,"(4-Methylphenyl)(diphenyl)sulfanium 4,4,5,5,6,...","(4-Methylphenyl)(diphenyl)sulfanium 4,4,5,5,6,...","(4-Methylphenyl)(diphenyl)sulfanium 4,4,5,5,6,...",862261-51-6,"(4-Methylphenyl)(diphenyl)sulfanium 4,4,5,5,6,..."
15406,DTXSID001046130,11beta-Hydroxysaxitoxin bis(trifluoroacetate),1923856-43-2,11beta-Hydroxysaxitoxin bis(trifluoroacetate) ...,11beta-Hydroxysaxitoxin bis(trifluoroacetate) ...,11beta-Hydroxysaxitoxin bis(trifluoroacetate),1923856-43-2,11beta-Hydroxysaxitoxin bis(trifluoroacetate)


In [6]:
df['query'] = df['Substance_Name'] + ' OR ' + df['Substance_CASRN']+" [tiab]"

In [10]:
df['Column3'] = df.apply(lambda row: row['Substance_Name'] + ' OR '+ row['Substance_CASRN'] if ' ' not in row['Substance_Name'] else f'{row["Substance_Name"]} OR {row["Substance_CASRN"]}', axis=1)

In [11]:
df['term1'] = [e.split("OR")[0] for e in df['Column3']]
df['term2'] = [e.split("OR")[1] for e in df['Column3']]

In [12]:
df['term3'] = [f"{e}" for e in df['term1']]

In [13]:
df['term1'] = [e.strip() for e in df['term1']]

In [16]:
df['term3'] = ['"' + s + '"' for s in df['term1']]

In [17]:
df['term4']= ['"' + s + '"' for s in df['term2']]

In [18]:
import time
import requests
import json

In [22]:
my_lst = []
#url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmode=json&retmax=1&sort=relevance&term=({term2} OR {term3})"
for i, row in df.iterrows():
    term, term1, term2,  = row['dtxsid'], row['term3'], row['term4']
   # print(term, term1, term2)
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmode=json&retmax=1&sort=relevance&term=({term1} OR {term2})[tiab]"
    #print(url)
    try:
        response = requests.get(url).json()
        a = response['esearchresult']['count']
        time.sleep(1)
    except Exception as error:
        print(error)
    finally:
        my_lst.append([term,a])

In [24]:
len(my_lst)

13041

In [28]:
pubmed = pd.DataFrame(my_lst, columns = ['dtxsid', 'article_count'])
#.to_csv(interim_dir+'pubmed_counts_110524csv')

In [31]:
pubmed['article_count'] = pubmed['article_count'].astype(int)

In [33]:
pubmed['Pubmed'] = pubmed['article_count'].apply(lambda x: 1 if x>= 1 else 0)

In [35]:
pubmed['Pubmed'].value_counts()

0    8344
1    4697
Name: Pubmed, dtype: int64

In [37]:
pubmed.to_csv(interim_dir+'pubmed_counts_110524.csv')